In [1]:
import pandas as pd
import gc

In [2]:
sales_train_validation = pd.read_csv('/project_data/data_asset/sales_train_validation.csv')
calendar = pd.read_csv('/project_data/data_asset/calendar.csv')
sell_prices = pd.read_csv('/project_data/data_asset/sell_prices.csv')
submission = pd.read_csv('/project_data/data_asset/sample_submission.csv')

In [3]:
validation_rows = [row for row in submission['id'] if 'validation' in row]
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row]
submission_validation = submission.loc[submission['id'].isin(validation_rows)]
submission_evaluation = submission.loc[submission['id'].isin(evaluation_rows)]

submission_validation.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921',
                                 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 
                                 'd_1931', 'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 
                                 'd_1940', 'd_1941']
submission_evaluation.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949',
                                 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 
                                 'd_1959', 'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 
                                 'd_1968', 'd_1969']

del submission
gc.collect()

7

In [4]:
products = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

In [5]:
submission_validation = submission_validation.merge(products, how = 'right', on = 'id')
submission_evaluation = submission_evaluation.merge(products, how = 'right', on = 'id')
gc.collect()

42

In [6]:
def melt_sales(df):
    return pd.melt(frame = df, 
                   id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                   var_name = 'day', 
                   value_name = 'demand')

In [7]:
submission_validation_melted = melt_sales(submission_validation)
submission_evaluation_melted = melt_sales(submission_evaluation)
gc.collect()

126

In [8]:
sales_train_melted = melt_sales(sales_train_validation)
print('Melted sales train validation has {} rows and {} columns'.format(sales_train_melted.shape[0], 
                                                                        sales_train_melted.shape[1]))
del sales_train_validation
gc.collect()

Melted sales train validation has 58327370 rows and 8 columns


70

In [ ]:
# Concatenate train, validation and test sets into one dataframe
sales_train_melted['partition'] = 'train'
submission_validation_melted['partition'] = 'validation'
submission_evaluation_melted['partition'] = 'evaluation'
full_data_melted = pd.concat([sales_train_melted, submission_validation_melted, submission_evaluation_melted], axis = 0)

In [ ]:
def merge_with_calendar(df, calendar):
    df_with_cal = pd.merge(df, calendar, how = "left", left_on = ["day"], right_on = ["d"])
    df_with_cal.drop(['d', 'day'], inplace = True, axis = 1)
    return df_with_cal

In [ ]:
# Merge sales data with calendar features
sales_train_melted = merge_with_calendar(sales_train_melted, calendar)
submission_validation_melted = merge_with_calendar(submission_validation_melted, calendar)
gc.collect()

In [ ]:
def merge_with_prices(df, sell_prices):
    df_with_prices = df.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    return df_with_prices

In [ ]:
sales_train_melted = merge_with_prices(sales_train_melted, sell_prices)
submission_validation_melted = merge_with_prices(submission_validation_melted, sell_prices)
gc.collect()